In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorflow import constant
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column 
from tensorflow import estimator
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from wandb.keras import WandbCallback
import wandb
import time
%matplotlib inline

Using TensorFlow backend.


In [2]:
tf.random.set_seed(67)
np.random.seed(67)

In [3]:
df = pd.read_csv("iris.csv")

In [4]:
X = np.array(df.drop('species',axis=1))
Y = np.array(df['species'])

In [5]:
Le = LabelEncoder()
Le.fit(['setosa', 'virginica', 'versicolor'])
Y_new = Le.transform(Y)

In [6]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, Y_new, test_size=0.30, random_state=101)

In [ ]:
labels = (['setosa', 'virginica', 'versicolor'])

In [7]:
METHOD = 'bayes'

In [12]:
def train():
    # Prepare data tuples
    (X_train, y_train) = X_train1, Y_train1
    (X_test, y_test) = X_test1, Y_test1
    
    # Default values for hyper-parameters we're going to sweep over
    configs = {
        'layers': 4,
        'batch_size': 64,
        'epochs': 5,
        'method': METHOD
    }
    
    # Initilize a new wandb run
    wandb.init(project='hyperparameter-sweeps-comparison', config=configs)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Define the model
    model_keras = Sequential()

    # Define the first layer
    model_keras.add(keras.layers.Dense(36, activation='relu', input_shape = (4,)))
    model_keras.add(keras.layers.Dropout(0.2))
    # Add activation function to classifier
    model_keras.add(keras.layers.Dense(18, activation='relu'))
    model_keras.add(keras.layers.Dropout(0.5))

    model_keras.add(keras.layers.Dense(3, activation='softmax'))
    
    # Compile the model
    model_keras.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model
    model_keras.fit(X_train, y_train, 
             epochs=config.epochs,
             batch_size=config.batch_size,
             validation_data=(X_test, y_test),
             callbacks=[WandbCallback(validation_data=(X_test, y_test), labels=labels)])

In [13]:
def get_sweep_id(method):
    sweep_config = {
        'method': method,
        'metric': {
          'name': 'accuracy',
          'goal': 'maximize'   
        },
        'parameters': {
            'layers': {
                'values': [4, 8, 16]
            },
            'batch_size': {
                'values': [32, 64, 96, 128]
            },
            'epochs': {
                'values': [5, 10, 15]
            }
        }
    }
    sweep_id = wandb.sweep(sweep_config, project='hyperparameter-sweeps-comparison')
    
    return sweep_id

In [14]:
sweep_id = get_sweep_id('bayes')

Create sweep with ID: rda1hnv0
Sweep URL: https://app.wandb.ai/sinisha123/hyperparameter-sweeps-comparison/sweeps/rda1hnv0


In [15]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 52w8nbh1 with config:
	batch_size: 128
	epochs: 5
	layers: 16
wandb: Agent Started Run: 52w8nbh1


Process Process-2:
Traceback (most recent call last):
  File "/home/sinisha/miniconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/sinisha/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sinisha/miniconda3/lib/python3.6/site-packages/wandb/wandb_agent.py", line 64, in _start
    function()
  File "<ipython-input-12-5261899e0f73>", line 42, in train
    callbacks=[WandbCallback(validation_data=(X_test, y_test), labels=labels)])
NameError: name 'labels' is not defined
wandb: Ctrl-c pressed. Waiting for runs to end. Press ctrl-c again to terminate them.
